# Project UAS AMS
#### Vincent Tanaka - A12.2020.06519
#### Topik :Rekomendasi Anime Menggunakan Document Similarity di Bagian Sinopsis

Notebook ini merupakan tahap tahap dalam membuat sistem rekomendasi anime berdasarkan dataset publik yang didapatkan. Dataset tersebut berisi list anime sampai tahun 2020. Lalu program berjalan dengan mencari berdasarkan metadata berkaitan dengan anime yang berbeda, program akan mencoba dan merekomendasikan animeserupa yang menarik!

Fokus program ini terletak pada NLP, dengan memanfaatkan sinopsis berbasis teks untuk setiap animeuntuk mencoba dan merekomendasikan anime serupa berdasarkan anime tertentu yang diminati. Ini termasuk dalam rekomendasi berbasis konten.

## import/install Library

In [1]:
import nltk
import pathlib
import pandas as pd
import numpy as np
import re
import contractions
pathlib.Path().resolve()

WindowsPath('C:/Users/Vincent/Documents/AMS-UAS')

In [2]:
df = pd.read_csv('C:/Users/Vincent/Documents/AMS-UAS/MyAnimeList-Database-master/data/anime_with_synopsis.csv',on_bad_lines='skip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16214 entries, 0 to 16213
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MAL_ID     16214 non-null  int64 
 1   Name       16214 non-null  object
 2   Score      16214 non-null  object
 3   Genres     16214 non-null  object
 4   sypnopsis  16206 non-null  object
dtypes: int64(1), object(4)
memory usage: 633.5+ KB


In [3]:
df.head(15)

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witches are individuals with special powers li...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
5,15,Eyeshield 21,7.95,"Action, Sports, Comedy, Shounen",Sena is like any other shy kid starting high s...
6,16,Hachimitsu to Clover,8.06,"Comedy, Drama, Josei, Romance, Slice of Life","Yuuta Takemoto, a sophomore at an arts college..."
7,17,Hungry Heart: Wild Striker,7.59,"Slice of Life, Comedy, Sports, Shounen",Kyosuke Kano has lived under the shadow of his...
8,18,Initial D Fourth Stage,8.15,"Action, Cars, Sports, Drama, Seinen",Takumi Fujiwara finally joins Ryousuke and Kei...
9,19,Monster,8.76,"Drama, Horror, Mystery, Police, Psychological,...","Dr. Kenzou Tenma, an elite neurosurgeon recent..."


In [4]:
df = df[['Name','sypnopsis', 'Score']]
df.dropna(inplace=True)
df = df.sort_values(by=['Score'], ascending=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16206 entries, 16213 to 2667
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       16206 non-null  object
 1   sypnopsis  16206 non-null  object
 2   Score      16206 non-null  object
dtypes: object(3)
memory usage: 506.4+ KB


In [5]:
df.head(20)

,Name,sypnopsis,Score
16213,Scarlet Nexus,Solar calendar year 2020: grotesque organisms ...,Unknown
9052,Genei Toshi,Independent animation by Shimamura Tatsuo.,Unknown
9248,Tokai no Butchy,me made by Tezuka Productions.,Unknown
9250,Geori-eui Mubeopja,"In the year 2010, after World War III, the wor...",Unknown
9253,Kkurukkuru-wa Chingu-deul,The story is set in a village called Green For...,Unknown
9254,Uli Chingu Kkachi,korean movie from 1996.,Unknown
9255,Cooking Papa Christmas Special,Special of Cooking Papa.,Unknown
9256,McDull: Xiang Dang Dang Ban Wo Xin,"Hong-Kong made fourth sequel of ""My Life as Mc...",Unknown
9257,McDull: Wo He Wo Mama,Hong-Kong made fifth animated work of the McDu...,Unknown
9259,Uli Sai Jjang-Iya,Korean movie from 2004.,Unknown


In [6]:
df = df[df["Score"].str.contains("Unknown") == False]

In [7]:
df.head(30)

,Name,sypnopsis,Score
3446,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",9.19
14647,Shingeki no Kyojin: The Final Season,Gabi Braun and Falco Grice have been training ...,9.17
4953,Steins;Gate,The self-proclaimed mad scientist Rintarou Oka...,9.11
5660,Hunter x Hunter (2011),Hunter x Hunter is set in a world where Hunter...,9.1
8879,Gintama°,"Gintoki, Shinpachi, and Kagura return as the f...",9.1
13720,Shingeki no Kyojin Season 3 Part 2,Seeking to restore humanity's diminishing hope...,9.1
5234,Gintama',"After a one-year hiatus, Shinpachi Shimura ret...",9.08
723,Ginga Eiyuu Densetsu,The 150-year-long stalemate between the two in...,9.07
6377,Gintama': Enchousen,"While Gintoki Sakata was away, the Yorozuya fo...",9.04
11735,3-gatsu no Lion 2nd Season,"Now in his second year of high school, Rei Kir...",9.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11090 entries, 3446 to 2667
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       11090 non-null  object
 1   sypnopsis  11090 non-null  object
 2   Score      11090 non-null  object
dtypes: object(3)
memory usage: 346.6+ KB


# text preprocessing 

In [9]:
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['sypnopsis']))
len(norm_corpus)

11090

## TFIDF-Extract

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(11090, 59506)

# Document Similarity / Cosine

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11080,11081,11082,11083,11084,11085,11086,11087,11088,11089
0,1.000000,0.007628,0.010528,0.013299,0.011189,0.011690,0.007259,0.036686,0.007098,0.007402,...,0.0,0.0,0.0,0.000000,0.008730,0.011101,0.032709,0.007939,0.003045,0.009747
1,0.007628,1.000000,0.011188,0.018875,0.016670,0.354333,0.011692,0.019507,0.012303,0.005041,...,0.0,0.0,0.0,0.016926,0.011988,0.000000,0.004756,0.020958,0.035263,0.010534
2,0.010528,0.011188,1.000000,0.000000,0.015392,0.002623,0.010844,0.014412,0.003416,0.006048,...,0.0,0.0,0.0,0.000000,0.008854,0.020381,0.020086,0.000000,0.012765,0.010207
3,0.013299,0.018875,0.000000,1.000000,0.008706,0.012320,0.005566,0.006737,0.008767,0.023371,...,0.0,0.0,0.0,0.000000,0.005399,0.005804,0.030833,0.003062,0.008790,0.005322
4,0.011189,0.016670,0.015392,0.008706,1.000000,0.008039,0.259764,0.011095,0.247648,0.010863,...,0.0,0.0,0.0,0.000000,0.000000,0.009150,0.016290,0.014190,0.008395,0.022133


In [12]:
animes_list = df['Name'].values
animes_list, animes_list.shape

(array(['Fullmetal Alchemist: Brotherhood',
        'Shingeki no Kyojin: The Final Season', 'Steins;Gate', ...,
        'Ex-Arm', 'Utsu Musume Sayuri', 'Tenkuu Danzai Skelter+Heaven'],
       dtype=object),
 (11090,))

# testing contoh rekomendasi

In [13]:
anime_idx = np.where(animes_list == 'Bakemonogatari')[0][0]
anime_idx

177

In [14]:
anime_similarities = doc_sim_df.iloc[anime_idx].values
anime_similarities

array([0.01059535, 0.00628366, 0.01620949, ..., 0.        , 0.        ,
       0.00441772])

In [15]:
similar_anime_idxs = np.argsort(-anime_similarities)[1:6]
similar_anime_idxs

array([567, 323,  16,  27,  23], dtype=int64)

In [16]:
similar_anime = animes_list[similar_anime_idxs]
similar_anime 

array(['Nekomonogatari: Kuro', 'Nisemonogatari',
       'Owarimonogatari 2nd Season', 'Monogatari Series: Second Season',
       'Kizumonogatari III: Reiketsu-hen'], dtype=object)

In [17]:
#Cari 3 Rekomendasi Anime
def anime_recommender(anime_titles, animes = animes_list, doc_sims = doc_sim_df) :
#Ambil ID/Nama Anime
    anime_idx = np.where(animes == anime_titles)[0][0]
#Dapatkan Nilai/Value Kesamaan/Similarity
    anime_similarities = doc_sim_df.iloc[anime_idx].values
#Dapatkan ID/Anime yang memiliki Similarity
    similar_anime_idxs = np.argsort(-anime_similarities)[1:6]
#Dapatkan 5 nilai Array dari Similarity
    similar_anime = animes_list[similar_anime_idxs]
#return value top 5 anime rekomendasi
    return similar_anime

# Carilah rekomendasi anime dengan list anime berikut 

In [18]:
List_anime  = ['Bakemonogatari', 'Steins;Gate', 'One Piece', 'Monster', 'Cowboy Bebop']
for anime in List_anime:
    print('List Anime:', anime)
    print('Top 5 rekomendasi anime:', anime_recommender(anime_titles = anime, animes = animes_list, doc_sims = doc_sim_df))
    print()

List Anime: Bakemonogatari
Top 5 rekomendasi anime: ['Nekomonogatari: Kuro' 'Nisemonogatari' 'Owarimonogatari 2nd Season'
 'Monogatari Series: Second Season' 'Kizumonogatari III: Reiketsu-hen']

List Anime: Steins;Gate
Top 5 rekomendasi anime: ['Steins;Gate 0' 'Steins;Gate Movie: Fuka Ryouiki no Déjà vu'
 'Steins;Gate: Oukoubakko no Poriomania'
 'Steins;Gate: Kyoukaimenjou no Missing Link - Divide By Zero'
 'Rikei ga Koi ni Ochita no de Shoumei shitemita.']

List Anime: One Piece
Top 5 rekomendasi anime: ['One Piece: Episode of East Blue - Luffy to 4-nin no Nakama no Daibouken'
 'One Piece: Romance Dawn Story' 'One Piece Film: Strong World'
 'One Piece: Oounabara ni Hirake! Dekkai Dekkai Chichi no Yume!'
 'One Piece Film: Strong World Episode 0']

List Anime: Monster
Top 5 rekomendasi anime: ['Astro Boy: Tetsuwan Atom' 'Jetter Mars' 'Junod'
 'Dragon Ball Z Movie 02: Kono Yo de Ichiban Tsuyoi Yatsu'
 'Wonder Beat Scramble']

List Anime: Cowboy Bebop
Top 5 rekomendasi anime: ['Ginga Senp

# Test menggunakan Word-Embedding/Penyematan Kata

In [19]:
from gensim.models import FastText

tokenized_docs = [doc.split() for doc in norm_corpus]
ft_model = FastText(tokenized_docs, window=30, min_count=2, workers=4, sg=1)

# Hasilkan output penyematan tingkat dokumen / Word Embedding.

In [20]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [21]:
doc_vecs_ft = averaged_word2vec_vectorizer(tokenized_docs, ft_model, 100)
doc_vecs_ft.shape

(11090, 100)

# Document Similarity / Cosinus
Gunakan cosinus lagi untuk menghasilkan rekomendasi

In [22]:
doc_sim = cosine_similarity(doc_vecs_ft)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,11080,11081,11082,11083,11084,11085,11086,11087,11088,11089
0,1.000000,0.941087,0.910634,0.908180,0.901330,0.942092,0.893675,0.932133,0.926367,0.844090,...,0.626291,0.743505,0.612841,0.720210,0.877620,0.856140,0.941965,0.907356,0.862266,0.919410
1,0.941087,1.000000,0.897546,0.902023,0.919364,0.982783,0.900298,0.942751,0.927503,0.850404,...,0.593788,0.722239,0.562894,0.733560,0.863813,0.849735,0.935278,0.882533,0.850442,0.932323
2,0.910634,0.897546,1.000000,0.902397,0.958344,0.899980,0.955566,0.862803,0.954651,0.901167,...,0.686961,0.800229,0.681840,0.773562,0.914912,0.926247,0.929651,0.945074,0.930104,0.922342
3,0.908180,0.902023,0.902397,1.000000,0.895411,0.908932,0.878110,0.876554,0.903159,0.876533,...,0.582343,0.678592,0.583624,0.697499,0.884679,0.870403,0.919746,0.882817,0.892344,0.896587
4,0.901330,0.919364,0.958344,0.895411,1.000000,0.924856,0.961359,0.872115,0.967171,0.872923,...,0.647216,0.777814,0.648937,0.790530,0.937874,0.916557,0.925629,0.933738,0.921412,0.922758


In [23]:
for anime in List_anime:
    print('List Anime:', anime)
    print('Top 5 rekomendasi anime:', anime_recommender(anime_titles = anime, animes = animes_list, doc_sims = doc_sim_df))
    print()

List Anime: Bakemonogatari
Top 5 rekomendasi anime: ['Owarimonogatari 2nd Season' 'Kizumonogatari I: Tekketsu-hen'
 'Nekomonogatari: Kuro' 'Nisemonogatari' 'Tsukimonogatari']

List Anime: Steins;Gate
Top 5 rekomendasi anime: ['Steins;Gate 0' 'Higashi no Eden' 'NHK ni Youkoso!'
 'Kara no Kyoukai: Mirai Fukuin' 'Kimi no Suizou wo Tabetai']

List Anime: One Piece
Top 5 rekomendasi anime: ['One Piece Film: Z' 'One Piece Movie 1' 'Rio: Rainbow Gate!'
 'Magi: Sinbad no Bouken' 'Takarajima']

List Anime: Monster
Top 5 rekomendasi anime: ['Junod' 'Kyoufu Densetsu Kaiki! Frankenstein' 'Argento Soma' 'Black Fox'
 'Virtua Fighter']

List Anime: Cowboy Bebop
Top 5 rekomendasi anime: ['Ginga Tetsudou Monogatari' 'Elysium' 'Seihou Bukyou Outlaw Star'
 'Cowboy Bebop: Tengoku no Tobira' 'Space Cobra']

